In [ ]:
import tensorflow

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
project_path='/content/drive/My Drive/Colab Notebooks/'

In [ ]:
import pandas as pd
data=pd.read_csv(project_path+'Sentiment-1.csv')

In [ ]:
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [ ]:
data=data[['text','sentiment']]

In [ ]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data=data[data.sentiment!="Neutral"]
data['text']=data['text'].apply(lambda x:x.lower())
data['text']=data['text'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]','',x))
print(data[data['sentiment']=='Positive'].size)
print(data[data['sentiment']=='Negative'].size)

4472
16986


In [ ]:
for i,row in data.iterrows():
  row[0]=row[0].replace('rt',' ')

In [ ]:
vocabsize=2000
tokenizer=Tokenizer(num_words=vocabsize,split=' ')
tokenizer.fit_on_texts(data['text'].values)
x=tokenizer.texts_to_sequences(data['text'].values)
x=pad_sequences(x)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,SpatialDropout1D

In [ ]:
embed_dim=128
lstm_out=196
model=Sequential()
model.add(Embedding(vocabsize,embed_dim,input_length=x.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
from sklearn.model_selection import train_test_split
y=pd.get_dummies(data['sentiment']).values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=1)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(9119, 28) (9119, 2)
(1610, 28) (1610, 2)


In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
285/285 [==============================] - 56s 182ms/step - loss: 0.4173 - accuracy: 0.8243
Epoch 2/10
285/285 [==============================] - 48s 168ms/step - loss: 0.3121 - accuracy: 0.8690
Epoch 3/10
285/285 [==============================] - 48s 167ms/step - loss: 0.2736 - accuracy: 0.8854
Epoch 4/10
285/285 [==============================] - 49s 171ms/step - loss: 0.2510 - accuracy: 0.8945
Epoch 5/10
285/285 [==============================] - 49s 171ms/step - loss: 0.2258 - accuracy: 0.9037
Epoch 6/10
285/285 [==============================] - 53s 186ms/step - loss: 0.2083 - accuracy: 0.9149
Epoch 7/10
285/285 [==============================] - 49s 170ms/step - loss: 0.1868 - accuracy: 0.9208
Epoch 8/10
285/285 [==============================] - 48s 170ms/step - loss: 0.1748 - accuracy: 0.9244
Epoch 9/10
285/285 [==============================] - 47s 165ms/step - loss: 0.1594 - accuracy: 0.9324
Epoch 10/10
285/285 [==============================] - 47s 164ms/step - l